In [89]:
from abc import ABC, abstractmethod
import hashlib
from pathlib import Path
import shutil

from py7zr import unpack_7zarchive
import requests


TMP_DIR = Path('../tmp/')


class DataSource(ABC):
    name = None
    checksum = None
    
    def __init__(self, *, name=None, checksum=None):
        if name is not None:
            self.name = name
        if checksum is not None:
            self.checksum = checksum
        assert self.name is not None, 'name is not defined'
    
    def run(self, force=False):
        self.target_dir.mkdir(parents=True, exist_ok=True)
        data_path = self.get(force)
        self.verify_checksum(data_path)
        self.unpack(data_path)
        self.arrange()
    
    @abstractmethod
    def get(self, force):
        ...
    
    def verify_checksum(self, data_path):
        CHUNK_SIZE = 1024 * 64
        hash_sha256 = hashlib.sha256()
        with open(data_path, 'rb') as f:
            for chunk in iter(lambda: f.read(CHUNK_SIZE), b''):
                hash_sha256.update(chunk)

        checksum = hash_sha256.hexdigest()
        if self.checksum is not None:
            assert self.checksum == checksum, (
                f'invalid checksum. expected {self.checksum} but got {checksum}'
            )
        else:
            print(f'checksum for {data_path} is {checksum}')
    
    def unpack(self, archive):
        ...
    
    @property
    def target_dir(self):
        return TMP_DIR / self.name
    
    def arrange(self):
        ...


class ArchiveDataSource(DataSource):
    
    def unpack(self, archive):
        # shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
        shutil.unpack_archive(filename=archive, extract_dir=self.target_dir)

        
class RemoteDataSource(ArchiveDataSource):
    url = None
    
    def __init__(self, *, url=None, **kwargs):
        super().__init__(**kwargs)
        if url is not None:
            self.url = url
        assert self.url is not None, 'url is not defined'
    
    def get(self, force):
        download_dest = TMP_DIR / Path(self.url).name
#         print(download_dest, download_dest.resolve())
#         print(download_dest.exists(), force)
        if not download_dest.exists() or force:
            print('fetching', self.url)
            response = requests.get(self.url)
            response.raise_for_status()
            with open(download_dest, 'wb') as f:
                f.write(response.content)
        return download_dest


class VeekunDataSource(RemoteDataSource):
    path_to_sprites = Path('pokemon') / 'main-sprites'
    
    def __init__(self, *, path_to_sprites=None, extra_moves=(), **kwargs):
        super().__init__(**kwargs)
        if path_to_sprites is not None:
            self.path_to_sprites = path_to_sprites
        self.extra_moves = extra_moves
        assert self.path_to_sprites is not None, 'path_to_sprites is not defined'
    
    def run(self, force=False):
        super().run(force)
        self.do_extra_moves()
    
    def arrange(self):
        for p in (self.target_dir / self.path_to_sprites).iterdir():
            if (TMP_DIR / p.name).exists():
                print('deleting existing', TMP_DIR / p.name)
                shutil.rmtree(TMP_DIR / p.name)
            shutil.move(str(p), str(TMP_DIR))  # NOTE: path-like objects are supported since python 3.9
        shutil.rmtree(self.target_dir)
    
    def do_extra_moves(self):
        for src, dst in self.extra_moves:
            if dst == None:
                shutil.rmtree(TMP_DIR / src)
            else:
                shutil.move(str(TMP_DIR / src), str(TMP_DIR / dst))
        

class BattlersDataSource(RemoteDataSource):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        try:
            shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
        except shutil.RegistryError as e:
            if '.7z is already registered for "7zip"' in str(e):
                pass
            else:
                raise
    
    def get(self, force):
        # TODO: Implement automatic download
        print('Please manually download the file from')
        print('  https://www.mediafire.com/folder/mi31mvoxx98ij/3D_Battlers')
        print('and save it as tmp/3D Battlers [All].7z')
        return TMP_DIR / '3D Battlers [All].7z'
        
    def arrange(self):
        TARGET_PATH = TMP_DIR / '3d-battlers-animated'

        if TARGET_PATH.exists():
            print('deleting existing', TARGET_PATH)
            shutil.rmtree(TARGET_PATH)
        # move all files in 'Front' into tmp/anim-3d-battlers
        shutil.move(str(self.target_dir / 'Front'), str(TARGET_PATH))
        shutil.rmtree(self.target_dir)

In [93]:
gen1_veekun = VeekunDataSource(
    name='generation-1', 
    url='https://veekun.com/static/pokedex/downloads/generation-1.tar.gz',
    checksum='2d0923f5abf1171b7e011b3ce9b879e8eee1fd56ec82dfbe597a2eafa63ca21c',
    extra_moves=(
        ('red-blue/back', None),
        ('red-blue/gray', None),
        ('red-green/back', None),
        ('red-green/gray', None),
        ('yellow/back', None),
        ('yellow/gray', None),
        ('yellow/gbc', 'yellow-gbc'),
    ),
)
gen2_veekun = VeekunDataSource(
    name='generation-2', 
    url='https://veekun.com/static/pokedex/downloads/generation-2.tar.gz',
    checksum='1a01266008cf726df5d273da96ec3cbbbd3da0f17bfada4b0b153a4c92b4517a',
    extra_moves=(
        ('gold/back', None),
        ('gold/shiny', None),
        ('silver/back', None),
        ('silver/shiny', None),
        ('crystal/back', None),
        ('crystal/shiny', None),
        ('crystal/animated', 'crystal-animated'),
    ),
)
gen3_veekun = VeekunDataSource(
    name='generation-3', 
    url='https://veekun.com/static/pokedex/downloads/generation-3.tar.gz',
    checksum='15b733baf9ef91fbde3ae957edb4d2ba75615601a515b41590ab87043370319c',
    extra_moves=(
        ('ruby-sapphire/back', None),
        ('ruby-sapphire/shiny', None),
        ('emerald/animated', 'emerald-animated'),
        ('emerald/frame2', 'emerald-frame2'),
        ('emerald/shiny', None),
        ('firered-leafgreen/back', None),
        ('firered-leafgreen/shiny', None),
    ),
)
gen4_veekun = VeekunDataSource(
    name='generation-4', 
    url='https://veekun.com/static/pokedex/downloads/generation-4.tar.gz',
    checksum='b1b69463aac872b54adf56f1159e8e6d2dfcbbecb7d71c7ebf832fe44140da41',
    extra_moves=(
        ('diamond-pearl/back', None),
        ('diamond-pearl/female', None),
        ('diamond-pearl/frame2', 'diamond-pearl-frame2'),
        ('diamond-pearl/shiny', None),
        ('platinum/back', None),
        ('platinum/female', None),
        ('platinum/frame2', 'platinum-frame2'),
        ('platinum/shiny', None),
        ('heartgold-soulsilver/back', None),
        ('heartgold-soulsilver/female', None),
        ('heartgold-soulsilver/frame2', 'heartgold-soulsilver-frame2'),
        ('heartgold-soulsilver/shiny', None),
    ),
)
gen5_veekun = VeekunDataSource(
    name='generation-5', 
    url='https://veekun.com/static/pokedex/downloads/generation-5.tar.gz',
    checksum='ee037a3319b2a6143c5c90f679be13a06126c2f5424e46023fe0f53d2631aa62',
    extra_moves=(
        ('black-white/back', None),
        ('black-white/female', None),
        ('black-white/shiny', None),
    ),
)

icons_veekun = VeekunDataSource(
    name='icons-unpacked', 
    url='https://veekun.com/static/pokedex/downloads/pokemon-icons.tar.gz',
    path_to_sprites=Path('pokemon'),
    checksum='f9850ce82d8e6e69c163112c47553458fd27805034217a5331a1ae12b2a1c8ac',
    extra_moves=(
        ('icons/female', None),
        ('icons/old', None),
        ('icons/right', None),
    ),
)
sugimori_veekun = VeekunDataSource(
    name='sugimori-unpacked', 
    url='https://veekun.com/static/pokedex/downloads/pokemon-sugimori.tar.gz',
    path_to_sprites=Path('pokemon'),
    checksum='9dcb5ab803725db99ec235df72da9cc20e96ac843d88394cff95a6b0bb06da16',
    extra_moves=(
        ('sugimori/female/521.png', 'sugimori/521--female.png'),
        ('sugimori/female/592.png', 'sugimori/592--female.png'),
        ('sugimori/female/593.png', 'sugimori/593--female.png'),
        ('sugimori/female/668.png', 'sugimori/668--female.png'),
        ('sugimori/female/', None),
    ),
)
dream_world_veekun = VeekunDataSource(
    name='dream-world-unpacked', 
    url='https://veekun.com/static/pokedex/downloads/pokemon-dream-world.tar.gz',
    path_to_sprites=Path('pokemon'),
    checksum='eaaf06ea99e71e34d8710f5cfd4923b8cd4d62f44124930afd02bc17046b6057',
    extra_moves=(
        ('dream-world/female/592.svg', 'dream-world/592--female.svg'),
        ('dream-world/female/593.svg', 'dream-world/593--female.svg'),
        ('dream-world/female/', None),
    ),
)

battlers = BattlersDataSource(
    name='3d-battlers-unpacked',
    url='https://www.mediafire.com/folder/mi31mvoxx98ij/3D_Battlers',
    checksum='a282265f827aaf309f08c1be7ea98726de14bca942823ea85e6d7c77338d1205',
)


# gen1_veekun.run()
# gen2_veekun.run()
# gen3_veekun.run()
# gen4_veekun.run()
# gen5_veekun.run()
# icons_veekun.run()
# sugimori_veekun.run()
dream_world_veekun.run()
# battlers.run()

deleting existing ../tmp/dream-world


In [ ]:
VEEKUN_URLS = (
    # Main Pokémon sprites
    'https://veekun.com/static/pokedex/downloads/generation-1.tar.gz',
    'https://veekun.com/static/pokedex/downloads/generation-2.tar.gz',
    'https://veekun.com/static/pokedex/downloads/generation-3.tar.gz',
    'https://veekun.com/static/pokedex/downloads/generation-4.tar.gz',
    'https://veekun.com/static/pokedex/downloads/generation-5.tar.gz',
    # Others
    'https://veekun.com/static/pokedex/downloads/pokemon-icons.tar.gz',
    # Other files
    'https://veekun.com/static/pokedex/downloads/pokemon-sugimori.tar.gz',
    'https://veekun.com/static/pokedex/downloads/pokemon-dream-world.tar.gz',
)

# 'https://www.mediafire.com/file/ri28x1uyc4xvy5q/3D_Battlers_%255BAll%255D.7z/file'
_3D_BATTERLS = 'https://download1589.mediafire.com/44gxo0kxjyug/ri28x1uyc4xvy5q/3D+Battlers+%5BAll%5D.7z'



In [ ]:
# verify checksum

In [2]:
with tarfile.open("../sprites/generation-1.tar.gz", "r:gz") as tar:
    print(tar.getmembers())

[<TarInfo 'pokemon/main-sprites/red-green' at 0x7fc33c56ca10>, <TarInfo 'pokemon/main-sprites/red-green/140.png' at 0x7fc33c56ce20>, <TarInfo 'pokemon/main-sprites/red-green/47.png' at 0x7fc33c56cd50>, <TarInfo 'pokemon/main-sprites/red-green/20.png' at 0x7fc33c56cef0>, <TarInfo 'pokemon/main-sprites/red-green/84.png' at 0x7fc33c622050>, <TarInfo 'pokemon/main-sprites/red-green/86.png' at 0x7fc33c622120>, <TarInfo 'pokemon/main-sprites/red-green/29.png' at 0x7fc33c6221f0>, <TarInfo 'pokemon/main-sprites/red-green/81.png' at 0x7fc33c6222c0>, <TarInfo 'pokemon/main-sprites/red-green/82.png' at 0x7fc33c622390>, <TarInfo 'pokemon/main-sprites/red-green/90.png' at 0x7fc33c622530>, <TarInfo 'pokemon/main-sprites/red-green/13.png' at 0x7fc33c622460>, <TarInfo 'pokemon/main-sprites/red-green/39.png' at 0x7fc33c622600>, <TarInfo 'pokemon/main-sprites/red-green/52.png' at 0x7fc33c6226d0>, <TarInfo 'pokemon/main-sprites/red-green/74.png' at 0x7fc33c6227a0>, <TarInfo 'pokemon/main-sprites/red-gree